# Porównanie 3 metod regresji na podstawie 3 różnych zbiorów danych

Autor:
Data:

W niniejszej pracy wykorzystuję metody:
- K najbliższych sąsiądów (KNN)
- Regresji liniowej
- Lasu losowego

Używam następujących zbiorów danych:
- Melbourne Housing
    - zmienna objaśniana: cena domu
- Wine Quality
    - zmienna objaśniana: jakość wina
- Abalone
    - zmienna objaśniana: wiek skamieliny

## Załadowanie potrzebnych bibliotek

In [1]:
import random
import os

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split, GridSearchCV


## Stałe

In [2]:
random_state = 0

np.random.seed(random_state)
os.environ["PYTHONHASHSEED"] = str(random_state)
random.seed(random_state)

## Wczytanie danych

In [ ]:
housing = pd.read_csv("data/melb_data.csv")
wine_quality = pd.read_csv("data/winequalityN.csv")
abalone = pd.read_csv("data/abalone.csv")

### Krótka analiza eksploracyjna danych

In [12]:
# jakiś wykresik itp.

## Preprocessing

In [ ]:
# To służy do wyboru kolumn chcesz wykonań daną transformację (np. imputacja brakujących danych)
imputer = SimpleImputer()
ct = ColumnTransformer()

### Podział na train i test

In [ ]:
# Jakie parametry wybrać tutaj? To trzeba wytłumaczyć jako zależne od wielkości próbki
X_train, X_test, y_train, y_test = train_test_split()

# Modelowanie

In [ ]:
linear_regression = LinearRegression(n_jobs=-1)
knn = KNeighborsRegressor(n_jobs=-1)
random_forest = RandomForestRegressor(n_jobs=-1, random_state=random_state)

In [1]:
# TODO: decide if we want to check anything here
linear_regression_params = {}

knn_params = {
    "n_neighbors": None,
    "weights": ["uniform", "distance"],
    "leaf_size": None,
    "p": [1, 2],
    # there could be other distance metrics but let's skip them
}

random_forest_params = {
    "n_estimator": None,
    "criterion": ["squared_error", "absolute_error"],
    "max_depth": [None, 3, 4, 5],
    "min_samples_split": None,
    "min_samples_leaf": None,
    "max_features": None,
    "bootstrap": None,
}

Przy wyczerpującym przeszukiwania siatki parametrów w celu znalezienia najlepszej kombinacji parametrów użyjemy walidacji krzyżowej .

[<img src="../img/grid_search_cross_validation.png" alt="drawing" width="400"/>]("../img/grid_search_cross_validation.png")
źródło: https://scikit-learn.org/stable/modules/cross_validation.html

In [10]:
# Możemy zrobić jeszcze poszukiwanie najlepszych hiperparametrów do modelu
grid_search_results = GridSearchCV()

## Analiza

Na tym tym posiadam już dostrojone, finalne modele. Teraz zostaną one porównane wg następujących metryk:

- MAE
- MAPE
- R2

In [4]:
# TODO: wybrać finalne metryki